# Assignment 1: Modeling Spikes

### Part 1: Imports and Properties

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.integrate import odeint

%matplotlib inline
plt.rcParams['figure.figsize'] = (48, 12)
plt.rcParams.update({'font.size': 36})

from ipywidgets import interact
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


## Part 2: LIF Model
Differential equations describe change in $V$ from state $i - 1$ to $i$ during time interval $dt$.

The LIF model is described by the following differential equation:

$$\tau_{m}\frac{dV}{dt} = -V_m(t) + R_mI(t)$$

Thus, we use the following update rule, performed during each time step $dt$.

$$V_m[i] = V_m[i-1] + \frac{(-V_m[i-1] + R_m*I[i-1]) * dt}{tau_m}$$

In [2]:
## Time Variables
t_total_lif = 100                    # total time to simulate (msec)
dt = 0.1                         # simulation time step (msec)
t_arr_lif = np.arange(0, t_total_lif, dt) # time array

## Constants
Cm = 10                         # capacitance (uF)
Rm = 1                          # resistance (kOhm)
tau_m = Rm*Cm                   # time constant (msec)
tau_ref = 4                     # refractory period (msec)
Vth = 1                         # spike threshold (V)
V_spike = 0.5                   # spike delta (V)

## Input/Outputs
I = 1.5                         # input current (A)

def lif_model(I, length):
    t_refr = 0                      # initial refractory time
    I_arr = np.zeros(len(t_arr_lif))    # dynamic input current over time
    I_start = 10                    # when to start input current (msec)

    for i in range(int(I_start / dt), int((I_start / dt) + (length / dt))):
        I_arr[i] = I

    Vm = np.zeros(len(t_arr_lif))       # potential (V) trace over time

    ## iterate over each time step
    for i, t in enumerate(t_arr_lif):
        if t > t_refr:
            Vm[i] = Vm[i-1] + ((-Vm[i-1] + I_arr[i-1]*Rm) * dt / tau_m)
            if Vm[i] >= Vth:
                Vm[i] += V_spike
                t_refr = t + tau_ref

    ## plot membrane potential trace
    plt.figure()

    color = 'tab:blue'
    ax = sns.lineplot(x=t_arr_lif, y=Vm, color=color)
    ax.set_title("LIF Neuron")
    ax.set_xlabel("Time (msec)")
    ax.set_ylabel("Membrane Potential (mV)", color=color)
    ax.tick_params(axis='y', labelcolor=color)
    ax.set_xlim([0, t_total_lif])
    ax.set_ylim([-0.01, 1.6])

    color = 'tab:red'
    ax2 = ax.twinx()
    sns.lineplot(x=t_arr_lif, y=I_arr, ax=ax2, color=color)
    ax2.set_ylabel("Input Current (A)", color=color)
    ax2.tick_params(axis='y', labelcolor=color)
    ax2.set_ylim([-0.01, 20])
    plt.show()
interact(lif_model, I=(0, 5, 0.1), length=(0, 90, 1));

interactive(children=(FloatSlider(value=2.0, description='I', max=5.0), IntSlider(value=45, description='lengt…

## Part 3: Izhikevich Model
Differential equations describe change in $V$ from state $i - 1$ to $i$ during time interval $dt$.

The Izhikevich model is described by the following differential equations:

$$\frac{dv}{dt} = 0.04v^2+5v+140-u+I$$

$$\frac{du}{dt} = a(bv-u)$$

Thus, we use the following update rules, performed during each time step $dt$.

$$u[i] = a(bv[i-1] - u[i-1])dt$$

$$v[i] = 0.04v[i-1]^2+5v[i-1]+140-u[i-1]+I[i-1]$$

In [3]:
## Time Variables
t_total_iz = 1000                    # total time to simulate (msec)
dt = 0.1                        # simulation time step (msec)
t_arr_iz = np.arange(0, t_total_iz, dt) # time array
t_refr = 0                      # initial refractory time

## Constants
a = 0.02                         # model parameter
b = 0.2                         # model parameter
c = -65                         # rest voltage (V)
d = 8                         # membrane recovery

v_th = 30
v_spike = 0.5

def iz_model(I, length):
    I_arr = np.zeros(len(t_arr_iz))
    I_start = 200
    for i in range(int(I_start / dt), int((I_start / dt) + (length / dt))):
        I_arr[i] = I

    v = c*np.ones(len(t_arr_iz))       # potential (V) trace over time
    u = np.zeros(len(t_arr_iz))

    ## Iterating over each time step
    for i in range(0, len(t_arr_iz) - 1):
        u[i+1] = u[i] + (a*(b*v[i] - u[i]) * dt)
        v[i+1] = v[i] + (0.04*(v[i]**2) + 5*v[i] + 140 - u[i] + I_arr[i])

        if v[i+1] >= v_th:
            v[i+1] = c
            u[i+1] += d

    plt.figure(figsize=(48, 12))

    color = 'tab:blue'
    ax = sns.lineplot(x=t_arr_iz, y=v, color=color)
    ax.set_title("Izhikevich Model")
    ax.set_xlabel("Time (msec)")
    ax.set_ylabel("Membrane Potential (mV)", color=color)
    ax.tick_params(axis='y', labelcolor=color)
    ax.set_xlim([0, t_total_iz])

    color = 'tab:red'
    ax2 = ax.twinx()
    sns.lineplot(x=t_arr_iz, y=I_arr, ax=ax2, color=color)
    ax2.set_ylabel("Input Current (A)", color=color)
    ax2.tick_params(axis='y', labelcolor=color)
    ax2.set_ylim([-0.01, 100])
    plt.show()
interact(iz_model, I=(0, 10, 0.1), length=(0, 800, 10));

interactive(children=(FloatSlider(value=5.0, description='I', max=10.0), IntSlider(value=400, description='len…

## Part 4: Hodgkin-Huxley Model
Differential equations describe change in $V$ from state $i - 1$ to $i$ during time interval $dt$.

The Hodgkin-Huxley model is described by the following differential equations:

$$C\frac{dv}{dt} = I-g_{Na}m^3h(V-V_{Na}) - g_Kn^4(V-V_K) - g_L(V-V_L)$$

$$C\frac{dm}{dt} = (1-m)a_m(V) - (m)b_m(V)$$

$$C\frac{dh}{dt} = (1-h)a_h(V) - (h)b_h(V)$$

$$C\frac{dn}{dt} = (1-n)a_n(V) - (n)b_n(V)$$

Thus, we use the following update rules, performed during each time step $dt$.

$$\Large \color{red}{\text{# TODO}}$$